<h1 style="color: #484848;font-size: 35px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">NLP Guide(Part 1): 📈EDA, text pre-processing &amp; Embeddings.</span></h1>

***


<blockquote><center><h2 style="color: #484848;font-size: 25px;font-weight: bold;font-family: monospace;padding-bottom:30px" >Introduction 📝</h2></center>
    <p style="font-size:16px;  font-family:monospace;">
        The Goal🥅 of this notebook is to explore text data in a clean beautiful manner.For this i am using 
        <a href='https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge',style="color:#159364; font-family:cursive;">Toxic Comment Classification Challenge</a>
        data.It is a multi-label classification problem, for detecting different types of toxicity like threats,obscenity,insults and identity-based hate.we will explore how imbalanced this dataset is,find correlations between labels and more.
I will show you some basic  text preprocessing techniques and understand pre-trained embeddings. 
        
   </p>
   <br>
  
<h3 style="color: #484848;font-size: 15px;font-weight: bold;font-family: monospace;" >Multi-label Classification:=</h3>
    <p style="font-size:16px;  font-family:monospace;">
     In multi-label classification tasks, each problem instance is associated with multiple classes at the same time.Emotion identification, image annotation, text categorization, semantic scene classification, or gene and protein function prediction are examples of such problems.There are strategies to solve multi-label classification problems.In which the first is converts the multi-label problem into multiple single-label classification problems and ignores the correlation among labels,the second is to consider the pair-wise correlation between labels, and the third is to looks at the high order correlation through a subset of labels.
    </p>

<h2 style="color: #484848;font-size: 25px;font-weight: bold;font-family: monospace;padding-bottom:30px" >Table of content</h2>
  
 
<ul style="font-size:16px;  font-family:monospace;">
    
  <li><a href='#1'>Data Loading 📂</a></li>
  <li><a href='#2'>EDA📈</a></li>
  <li><a href='#3'>WordClouds☁️</a></li>
  <li><a href='#4'>Pre-processing⚒️</a></li>
  <li><a href='#5'>Word vector</a></li>
  <li><a href='#6'>Glove Embedding</a></li>
  <li><a href='#7'>Model Training🏋️</a></li>
  <li><a href='#8'>References</a></li>
  
    
</ul>
</blockquote>

# <p style="color:red;; font-family:newtimeroman; font-size:80%;padding:100px; text-align:center; ">Disclaimer: the dataset used in this notebook contains text that may be considered profane, vulgar, or offensive.</p>

In [ ]:
import sys
import random
import operator
import zipfile
from time import sleep
import re
import gc
import math
import os
import time
import string
import random
import pickle

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from plotly import graph_objs as go
from IPython.display import display_html,HTML, Javascript

import tensorflow as tf 
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
from plotly.subplots import make_subplots

from PIL import Image

from tqdm.autonotebook import tqdm




import nltk
from pandas import DataFrame

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
# from cuml.feature_extraction.text import CountVectorizer
# from cuml.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from colorama import Fore, Back, Style
from wordcloud import WordCloud,STOPWORDS
from PIL import Image
from nltk.tokenize import word_tokenize

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation,BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model,load_model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

nltk.download('stopwords')

from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
import tensorflow as tf
from pprint import pprint


from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [ ]:
# Defining all our palette colours.
primary_azure = "#8A2BE2"
primary_slate = "#CCFFFF"
primary_aquamarine = "#7FFFD4"
primary_crimson = '#1E90FF'
primary_venom_green = "#728C00"
primary_orchid = "#D8BFD8"
primary_blue = "#496595"
primary_blue2 = "#85a1c1"
primary_blue3 = "#3f4d63"

primary_green = px.colors.qualitative.Plotly[2]

# Defining all colors for text.
HEAD = '\x1b[1;31;47m'
COMMENT1='\x1b[1;30;43m'
COMMENT2='\x1b[1;30;44m'

sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'#234E70'})
pd.set_option('display.max_columns',10)
sys.setrecursionlimit(10**8)

BASE_PATH = '../input/jigsaw-toxic-comment-classification-challenge'
unzip = zipfile.ZipFile('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
unzip.extractall()
unzip = zipfile.ZipFile('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
unzip.extractall()
unzip = zipfile.ZipFile('../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
unzip.extractall()

<a id='1'></a>
<div class="alert alert-block alert-info" style="font-size:14px; font-family:cursive;">
    <h1 style="color: #484848;font-size: 35px;font-weight: bold;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear"><center>Data Loading📂</center></span></h1>
</div>

In [ ]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
train_df.head()

In [ ]:
train_df['text_len'] = train_df['comment_text'].apply(lambda x: len(x.split(' ')))
test_df['text_len'] = test_df['comment_text'].apply(lambda x: len(x.split(' ')))

colors = [primary_slate,primary_aquamarine,primary_azure,primary_crimson,primary_venom_green,primary_green]
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_df['none'] = 1-train_df[label_cols].max(axis=1)
label_with_clean = label_cols.copy()
label_with_clean.append('none')

train_df['comment_text'].fillna("unknown", inplace=True)
test_df['comment_text'].fillna("unknown", inplace=True)
print(HEAD+"train data \n"+COMMENT1+"total data entries: \t"+COMMENT2+f"{len(train_df)}"+COMMENT1 +"\nmaximum comment len: \t"\
      +COMMENT2 +f"{max(train_df['text_len'])}"+COMMENT1+"\nminimum comment len: \t"+COMMENT2+f"{min(train_df['text_len'])}"+\
      COMMENT1 +"\nmean comment len: \t"+COMMENT2 +f"{train_df['text_len'].mean()}\n")

print(HEAD+f"test data \n"+COMMENT1+"total data entries: \t"+COMMENT2+f"{len(test_df)}"+COMMENT1 +"\nmaximum comment len: \t" \
      +COMMENT2 +f"{max(test_df['text_len'])}"+ COMMENT1 +"\nminimum comment len: \t"+COMMENT2+f"{min(test_df['text_len'])}"+ COMMENT1 +"\nmean comment len: \t"+ \
      COMMENT2 +f"{test_df['text_len'].mean()}")

print(HEAD+f"\nClean Vs. Toxic \n"+COMMENT1+"clean comments: \t"+COMMENT2+f"{len(train_df[train_df.none==1])}" \
     +COMMENT1+"\nAll kind of toxic comments: \t"+COMMENT2+f"{len(train_df[train_df.none==0])}")

print(HEAD+f"\nAll kind of toxicity \n"+COMMENT1+"toxic: \t\t"+COMMENT2+f"{len(train_df[train_df.toxic==1])}"\
     +COMMENT1+f"\nsevere toxic: \t"+COMMENT2+f"{len(train_df[train_df.severe_toxic==1])}"\
     +COMMENT1+f"\nobscene: \t"+COMMENT2+f"{len(train_df[train_df.obscene==1])}"\
     +COMMENT1+f"\nthreat: \t"+COMMENT2+f"{len(train_df[train_df.threat==1])}"\
     +COMMENT1+f"\ninsult: \t"+COMMENT2+f"{len(train_df[train_df.insult==1])}"\
     +COMMENT1+f"\nidentity hate: \t"+COMMENT2+f"{len(train_df[train_df.identity_hate==1])}")

<a id='2'></a>
<div class="alert alert-block alert-info" style="font-size:14px; font-family:cursive;">
    <h1 style="color: #484848;font-size: 35px;font-weight: bold;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear"><center>EDA📈</center></span></h1>
</div>

<p style="font-size:16px;  font-family:monospace;">
There is not a lot of difference between size of train and test data.but data is highly imbalanced,almost 90% comments are clean and only 10% comments have some kind of toxicity.
looking at the insights above we can say that the comments are holding multiple tags, we will see insights about this but first let's check some comments of all tags and try to determine how they categorized.
</p>

In [ ]:
print(HEAD+"clean comment:\n")
print(COMMENT1+train_df[train_df.none==1].iloc[3,1])

In [ ]:
print(HEAD+"toxic comment:\n")
print(COMMENT1+train_df[train_df.toxic==1].iloc[3,1])

In [ ]:
print(HEAD+'severe_toxic:\n')
print(COMMENT1+train_df[train_df.severe_toxic==1].iloc[2,1])

In [ ]:
print(HEAD+'threat:\n')
print(COMMENT1+train_df[train_df.threat==1].iloc[7,1])

In [ ]:
print(HEAD+'insult:\n')
print(COMMENT1+train_df[train_df.insult==1].iloc[22,1])

In [ ]:
print(HEAD+"Obscene:\n")
print(COMMENT1+train_df[train_df.obscene==1].iloc[2,1])

In [ ]:
print(HEAD+"identity_hate:\n")
print(COMMENT1+train_df[train_df.identity_hate==1].iloc[3,1])

In [ ]:
balance_counts = train_df[label_with_clean].sum().tolist()
balance_sum = pd.DataFrame(balance_counts, columns=['toxicity'],index=label_with_clean)
balance_sum.drop(['none'],inplace=True)

In [ ]:
fig = go.Figure()
for i,label in enumerate(zip(label_with_clean,colors)):
    fig.add_trace(go.Bar(
    x=[label[0]],
    y=[balance_counts[i]],
    name=label[0],
    text=[balance_counts[i]],
    textposition='auto',
    marker_color=label[1]
    ))
fig.update_layout(
    paper_bgcolor='#234E70',
    plot_bgcolor='rgba(0,0,0,0)',
    title='<span style="font-size:32px; font-family:Times New Roman">Dataset distribution by labels</span>',
    font=dict(color='white',family="Courier New, monospace"),
)
fig.show()


fig1 = px.pie(train_df['none'].value_counts().reset_index(), values='none',names=['non-toxic',"All type of toxicity"])
fig1.update_traces(textposition="inside", textinfo='percent+label')
fig1.update_layout(paper_bgcolor='#234E70',
    plot_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white',family="Courier New, monospace"),
    title='<span style="font-size:32px; font-family:Times New Roman">Distribution by very bad amd normal comments</span>',)
fig1.show()

fig1 = px.pie(balance_sum,values='toxicity',names=["toxic",'severe_toxic','obscene',"threat","insult","identity_hate"])
fig1.update_traces(textposition="inside", textinfo="percent+label")
fig1.update_layout(paper_bgcolor='#234E70',
    plot_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white',family="Courier New, monospace"),
    title='<span style="font-size:32px; font-family:Times New Roman">Distribution between types of toxic comments</span>',)
fig1.show()

In [ ]:
row_sums = train_df[label_cols].sum(axis=1)
x = row_sums.value_counts()


fig = go.Figure()
for i,label in enumerate(x):
    fig.add_trace(go.Bar(
        x=[i],
        y=[label],
        name=i,
        text=[label],
        textposition='auto',
        marker_color=[colors[5]]
    ))
fig.update_layout(
    paper_bgcolor='#234E70',
    plot_bgcolor='rgba(0,0,0,0)',
    title='<span style="font-size:32px; font-family:Times New Roman">Comments with multiple labels</span>',
    font=dict(color='white',family="Courier New, monospace"),
)
fig.show()    

<blockquote>
<h3 style="color: #484848;font-size: 20px;font-weight: bold;font-family: monospace;" >Correlation between lab:=</h3>
<p style="font-size:16px;  font-family:monospace;">
looking at the chart above we can see multiple entries are holding multiple tags and that is the normal behavior of multilabel classification problems because of that tags have correlation with other tags,for instance a comment can be type of toxic,severe-toxic and obscene.lets visualize correlation between labels.
</p>
</blockquote>

In [ ]:
def cutout(seq, idx):
    return seq[idx+1:] + seq[:idx]

corr_df = []
for i,main_cols in enumerate(zip(train_df.columns[2:8],colors)):
    corr_mats = []
    for other_cols in cutout(label_cols,i):
        confusion_matrix = pd.crosstab(train_df[main_cols[0]],train_df[other_cols])
        corr_mats.append(confusion_matrix)
    out = pd.concat(corr_mats,axis=1,keys=cutout(label_cols,i))
    out = out.style.highlight_min(color=colors[i], axis=0)
    out = out.set_table_styles([{'selector':'th','props':[('background-color','#234E70'),('color', 'cyan')]}])
    corr_df.append(out)

In [ ]:
df1_styler = corr_df[0].set_table_attributes("style='display:inline;margin:40px;font-family:monospace;font-style: oblique;font-weight: bold;font-variant: small-caps;font-size: 16px;color:black;'").set_caption('Correlation between toxic and other labels')
df2_styler = corr_df[1].set_table_attributes("style='display:inline;margin:40px;font-family:monospace;font-style: oblique;font-weight: bold;font-variant: small-caps;font-size: 16px;color:black;'").set_caption('Correlation between severe-toxic and other labels')
df3_styler = corr_df[2].set_table_attributes("style='display:inline;margin:40px;font-family:monospace;font-style: oblique;font-weight: bold;font-variant: small-caps;font-size: 17px;color:black;'").set_caption('Correlation between obscene and other labels')
df4_styler = corr_df[3].set_table_attributes("style='display:inline;margin:40px;font-family:monospace;font-style: oblique;font-weight: bold;font-variant: small-caps;font-size: 17px;color:black;'").set_caption('Correlation between threat and other labels')
df5_styler = corr_df[4].set_table_attributes("style='display:inline;margin:40px;font-family:monospace;font-style: oblique;font-weight: bold;font-variant: small-caps;font-size: 16px;color:black;'").set_caption('Correlation between insult and other labels')
df6_styler = corr_df[5].set_table_attributes("style='display:inline;margin:40px;font-family:monospace;font-style: oblique;font-weight: bold;font-variant: small-caps;font-size: 16px;color:black;'").set_caption('Correlation between idendity-hate and other labels')

display_html(df1_styler._repr_html_(), raw=True)
display_html(df2_styler._repr_html_(), raw=True)
display_html(df3_styler._repr_html_(), raw=True)
display_html(df4_styler._repr_html_(), raw=True)
display_html(df5_styler._repr_html_(), raw=True)
display_html(df6_styler._repr_html_(), raw=True)

<blockquote>
<h3 style="color: #484848;font-size: 20px;font-weight: bold;font-family: monospace;" >Text data noises:=</h3>
<p style="font-size:16px;  font-family:monospace;">
Text data contains many kind of impurities and noises that is build false correlations between labels.How to deal with these impurities totally depends on our approach to solve the problem, i will show you two approachs to deal with theses impurities in this notebook,but first we visualize thses noises.they are just punctuations('$',"'",',','&') ,urls, stopwords('the','i','are','and') words that do not give much insights about data and common in all instances.
</p>
</blockquote>

In [ ]:
stop_words = stopwords.words('english')
more_stop_words = ['u','im','c','ur']
stop_words = stop_words + more_stop_words

#unique_word_count 
train_df["unique_word"] = train_df["comment_text"].apply(lambda x:len(set(str(x).split())))
test_df["unique_word"] = train_df["comment_text"].apply(lambda x:len(set(str(x).split())))

# stop_word_count
train_df["stop_word"] = train_df['comment_text'].apply(lambda x:len([w for w in str(x).lower().split() if w in stop_words]))
test_df["stop_word"] = test_df['comment_text'].apply(lambda x:len([w for w in str(x).lower().split() if w in stop_words]))

# url_count
train_df['url_count'] = train_df['comment_text'].apply(lambda x:len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
test_df['url_count'] = test_df['comment_text'].apply(lambda x:len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# char_count
train_df['char_count'] = train_df['comment_text'].apply(lambda x: len(str(x)))
test_df['char_count'] = test_df['comment_text'].apply(lambda x: len(str(x)))

# punctuation_count
train_df['punctuation_count'] = train_df['comment_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test_df['punctuation_count'] = test_df['comment_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

In [ ]:
labels_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
balance_counts = train_df[labels_cols].sum().tolist()

def nano_method(num):
    if num%2==0:return 2
    else:return 1
    
dfs={}
for label in labels_cols:
    dfs[label] = train_df[train_df[label]==1]

fig = make_subplots(rows=3,cols=2)

for i, label in enumerate(labels_cols,1):
        fig.add_trace(
            go.Scatter(x=dfs[label][[label,'text_len']].index,
                       y=dfs[label][[label,'text_len']].values[:,1],name=label,fill='tozeroy'),
            row=math.ceil(i/2),col=nano_method(i)
        )

fig.update_layout(height=800, width=800,
    paper_bgcolor='#234E70',
    plot_bgcolor='rgba(0,0,0,0)',
    title='<span style="font-size:32px; font-family:Times New Roman">Data Roles in Different Fields</span>',
    font=dict(color='white',family="Courier New, monospace"),
    
)
fig.update_xaxes(range=[0,160000])
fig.update_yaxes(range=[0,2500])
fig.show()

In [ ]:
METAFEATURES = ['text_len','unique_word', 'stop_word','url_count','char_count','punctuation_count']

fig, axes = plt.subplots(ncols=2, nrows=len(METAFEATURES), figsize=(25,40),dpi=100)


for i, feature in tqdm(enumerate(METAFEATURES),desc='loop'):
    sns.histplot(dfs['toxic'][feature], label='toxic', ax=axes[i][0], color=colors[5],kde=True,common_norm=True)
    sns.histplot(dfs['severe_toxic'][feature], label='severe_toxic',ax=axes[i][0], color=colors[4],kde=True,common_norm=True)
    sns.histplot(dfs['obscene'][feature], label='obscene', ax=axes[i][0], color=colors[3],kde=True,common_norm=True)
    sns.histplot(dfs['threat'][feature], label='threat', ax=axes[i][0], color=colors[2],kde=True,common_norm=True)
    sns.histplot(dfs['insult'][feature], label='insult', ax=axes[i][0], color=colors[1],kde=True,common_norm=True)
    sns.histplot(dfs['identity_hate'][feature], label='identity_hate', ax=axes[i][0], color=colors[0],kde=True,common_norm=True)

    sns.histplot(train_df[feature], label="Training", ax=axes[i][1],color=colors[2],kde=True,common_norm=True)
    sns.histplot(test_df[feature], label='Test', ax=axes[i][1],color=colors[0],common_norm=True)
    sleep(0.01)
    for j in range(2):
        axes[i][j].set_xlabel('')
        axes[i][j].set_yscale('symlog', base=5,)
        axes[i][j].tick_params(axis='x', labelsize=20,colors='w')
        axes[i][j].tick_params(axis='y', labelsize=20,colors='w')
        axes[i][j].legend()
    
    axes[i][0].set_title(f"{feature} Target Distribution in Training Set", fontsize=20,color='white')
    axes[i][1].set_title(f"{feature} Training & Test Set Distribution", fontsize=20,color='white')
plt.show()
         

<a id='3'></a>
<div class="alert alert-block alert-info" style="font-size:14px; font-family:cursive;">
    <h1 style="color: #484848;font-size: 35px;font-weight: bold;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear"><center>WordClouds☁️</center></span></h1>
</div>


In [ ]:
stopword = set(STOPWORDS)

In [ ]:
#clean comments
clean_mask=np.array(Image.open("../input/logogo/Descarga gratis Mensaje.png"))
clean_mask=clean_mask[:,:,1]
#wordcloud for clean comments
subset=train_df[train_df.none==True]
text=subset.comment_text.values
wc= WordCloud(background_color="#234E70",max_words=2000,mask=clean_mask,stopwords=stopword)
wc.generate(" ".join(text))
plt.figure(figsize=(20,10))
plt.axis("off")
plt.title('Top words for clean comments', 
          fontsize=25,color='white')
plt.imshow(wc.recolor(colormap= 'viridis' , random_state=17), alpha=0.98)
plt.show()

In [ ]:
#toxic
plt.figure(figsize=(40,50))
plt.subplot(321)
toxic_mask = np.array(Image.open('../input/mzzzzzz/Negative Grade.jpeg'))
# toxic_mask = toxic_mask[:,:,1]
# words cloud for toxic comments
text = dfs['toxic'].comment_text.values
wc = WordCloud(background_color='black',max_words=500,mask=toxic_mask,stopwords=stopword)
wc.generate(" ".join(text))
plt.title('Words frequented in Toxic Comments', fontsize=25,color='black')
plt.axis('off')
plt.imshow(wc)



#severe_toxic
plt.subplot(322)
severe_toxic = np.array(Image.open("../input/masks-for-masks/Cat What Murderous Black Cat With Knife Halloween Sticker.jpeg"))
# severe_toxic = severe_toxic[:,:,1]
# wprds cloud for severe_toxic comments
text = dfs['severe_toxic'].comment_text.values
wc = WordCloud(background_color='black', max_words=500, mask=severe_toxic, stopwords=stopword)
wc.generate(" ".join(text))
plt.axis('off')
plt.title('Words frequented in severe toxic comments',fontsize=25,color='black')
plt.imshow(wc.recolor(colormap='Paired_r', random_state=122), alpha=0.96)

#obscene
plt.subplot(323)
obscene_mask = np.array(Image.open('../input/masks-for-masks/Image about cute in anime by socoki on We Heart It.jpeg'))
# obscene_mask = obscene_mask[:,:,1]
# words cloud for objscene comments
text = dfs['obscene'].comment_text.values
wc = WordCloud(background_color='black', max_words=500, mask=obscene_mask,stopwords=stopword)
wc.generate(" ".join(text))
plt.axis('off')
plt.title('Words frequented in obscene commnets', fontsize=25,color='black')
plt.imshow(wc.recolor(colormap='RdGy', random_state=215), alpha=0.96)

#threat
plt.subplot(324)
threat_mask = np.array(Image.open('../input/masks-for-masks/dinner Tosha C_ on ArtStation at https __www.artstation.jpeg'))
# threat_mask = threat_mask[:,:,1]
# words cloud for objscene comments
text = dfs['threat'].comment_text.values
wc = WordCloud(background_color='black', max_words=500, mask=threat_mask,stopwords=stopword)
wc.generate(" ".join(text))
plt.axis('off')
plt.title('Words frequented in threat commnets',fontsize=25,color='black')
plt.imshow(wc.recolor(colormap='rainbow', random_state=215), alpha=0.96)

#insult
plt.subplot(325)
insult_mask = np.array(Image.open('../input/mzzzzzz/Marshmello Dj Art Print by Shootme - X-Small.jpeg'))
# insult_mask = insult_mask[:,:,1]
# words cloud for objscene comments
text = dfs['insult'].comment_text.values
wc = WordCloud(background_color='black', max_words=200, mask=insult_mask,stopwords=stopword)
wc.generate(" ".join(text))
plt.axis('off')
plt.title('Words frequented in insult commnets',fontsize=25,color='black')
plt.imshow(wc.recolor(colormap='plasma', random_state=215), alpha=0.96)

#identity hate
plt.subplot(326)
identity_hate_mask = np.array(Image.open('../input/masks-for-masks/Bloodwraith A Fantasy World of Survival and Torment.jpeg'))
# identity_hate_mask = identity_hate_mask[:,:,1]
# words cloud for objscene comments
text = dfs['identity_hate'].comment_text.values
wc = WordCloud(background_color='black', max_words=600, mask=identity_hate_mask,stopwords=stopword)
wc.generate(" ".join(text))
plt.axis('off')
plt.title('Words frequented in identity hate commnets', fontsize=25,color='black')
plt.imshow(wc.recolor(random_state=125), alpha=0.96)

plt.show()

<a id='4'></a>
<div class="alert alert-block alert-info" style="font-size:14px; font-family:cursive;">
    <h1 style="color: #484848;font-size: 35px;font-weight: bold;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear"><center>Pre-processing⚒️</center></span></h1>
</div>
<br>
<p style="font-size:16px;  font-family:monospace;">
    Preprocessing is one of the major steps when we are dealing with any kind of text models.During this stage we have to look at the distribution of our data, what techniques are needed and how deep we should clean. Few preprocessing steps are converting to lowercase, removing punctuation, removing stop words and stemming/lemmatization.
    In this notebook we will be train our model on pre-trained embeddings,<span style='font-weight:bold'>and remember when we train model by pre-trained embedings we should not follow normal pre-processing steps</span> because of doing this we lost lots of useful imformations.we will see why is that when we learn about word embeddings in below cells.
</p>

<h2 style="color: #484848;font-size: 25px;font-weight: bold;font-family: monospace;" >Lowercase=></h2>
<p style="font-size:16px;  font-family:monospace;">
Programming languages consider textual data as sensitive, which means that <span style="font-weight:bold">Different</span> is different from <span style="font-weight:bold">different</span>. we humans know that those both belong to same token but due to the character encoding those are considered as different tokens. Converting to lowercase is a very mandatory preprocessing step.
</p>

<h2 style="color: #484848;font-size: 25px;font-weight: bold;font-family: monospace;" >Punctuation=></h2>

<p style="font-size:16px;  font-family:monospace;">
    Punctuation are the unnecessary symbols that are in our corpus documents, we should be little careful with what we are doing with this. There might be few problems such as U.S — us “United Stated” being converted to “us” after the preprocessing.
 </p>
 
<h2 style="color: #484848;font-size: 25px;font-weight: bold;font-family: monospace;" >Stopwords=></h2>

<p style="font-size:16px;  font-family:monospace;">
    Stopwords are the most commonly occuring words which don't give any valuable insights.And removing them normally gives better results and improves computations.
 </p>
 
 <h2 style="color: #484848;font-size: 25px;font-weight: bold;font-family: monospace;" >Lemmatisation=></h2>
 
 <p style="font-size:16px;  font-family:monospace;">
    Lemmatisation is a way to reduce the word to root synonym of a word. Unlike Stemming, Lemmatisation makes sure that the reduced word is again a dictionary word (word present in the same language). WordNetLemmatizer can be used to lemmatize any word.
 </p>
 
 <h2 style="color: #484848;font-size: 25px;font-weight: bold;font-family: monospace;" >Stemming=></h2>
 
 <p style="font-size:16px;  font-family:monospace;">
    Stemming reduces the word to its stem, meaning that playing and played are the same words which basically indicate an action play, so we convert playing and played to play. <span style="font-weight:bold">it is very importent normal pre-processing but equally bad for pre-processing for glove embedding. </span>
 </p>

In [ ]:
def remove_punctuation(text):
    punclist = string.punctuation + string.digits
    # remove punctuation from text
    table_ = str.maketrans(punclist,' '*len(punclist))
    text_list = ' '.join(text.translate(table_).split())
    return text_list

def remove_stopwords(text):
    stop_words = stopwords.words('english')
    more_stopwords = ['u','ya','yo','imma','im','c','da','be','s','on','off','na','nah','noo']
    stop_words = stop_words + more_stopwords
    text = ' '.join(word for word in text.split(' ') if word not in stop_words)
    return text

def remove_apostrophe(text):
    return text.replace("'","")
    
def stem_word(text):
    stemmer = PorterStemmer()
    text = ' '.join(stemmer.stem(word) for word in text.split(' '))
    return text

def lemm_word(text):
    lemmer = WordNetLemmatizer()
    text = ' '.join(lemmer.lemmatize(word) for word in text.split(' '))
    return text

In [ ]:
def standard_preprocess(text):
    text = str(text).lower()
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = remove_apostrophe(text)
    text = lemm_word(text)
    text = stem_word(text)
    text = remove_punctuation(text)
    return text

In [ ]:
%%time
tqdm.pandas()
train_df['simple_preprocess'] = train_df['comment_text'].progress_apply(standard_preprocess)
test_df['simple_preprocess'] = test_df['comment_text'].progress_apply(standard_preprocess)

<a id='5'></a>
<div class="alert alert-block alert-info" style="font-size:14px; font-family:cursive;">
    <h1 style="color: #484848;font-size: 35px;font-weight: bold;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear"><center>Word vector</center></span></h1>
</div>
<br>

<p style="font-size:16px;  font-family:monospace;">
    Word vectors represent a significant leap forward in advancing our ability to analyze relationships across words, sentences, and documents. In doing so, they advance technology by providing machines with much more information about words than has previously been possible using traditional representations of words. It is word vectors that make technologies such as speech recognition and machine translation possible.
 <br>  
   &nbsp&nbsp&nbsp&nbsp&nbsp word vectors represent words as multidimensional continuous floating point numbers where semantically similar words are mapped to proximate points in geometric space. In simpler terms, a word vector is a row of real-valued numbers (as opposed to dummy numbers) where each point captures a dimension of the word's meaning and where semantically similar words have similar vectors. This means that words such as wheel and engine should have similar word vectors to the word car (because of the similarity of their meanings), whereas the word banana should be quite distant. example here is showing that by using word vectors we can determine that:
     <br>
        <br>
   &nbsp&nbsp&nbsp&nbsp&nbsp<span style="font-weight:bold;">king - man + woman = queen</span>

 </p>


<a id='6'></a>
<div class="alert alert-block alert-info" style="font-size:14px; font-family:cursive;">
    <h1 style="color: #484848;font-size: 35px;font-weight: bold;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear"><center>Glove embedding</center></span></h1>
</div>
<br>

<p style="font-size:16px;  font-family:monospace;">
    GloVe method is built on an important idea,
    <br>
    <br>
    <blockquote>
        <span style="color: #484848;font-size: 20px;font-weight: 50px;font-family: monospace; letter-spacing: 2px;cursor: pointer">
           "You can derive semantic relationships between words from the co-occurrence matrix"
         </span>
     </blockquote>
    <br>
    <br>
    </p>
    <p style="font-size:16px;  font-family:monospace;">
    Word embeddings are basically a form of word representation that bridges the human understanding of language to that of a machine. They have learned representations of text in an n-dimensional space where words that have the same meaning have a similar representation. Meaning that two similar words are represented by almost similar vectors that are very closely placed in a vector space.
    <br>
    &nbsp&nbsp&nbsp&nbsp&nbspGiven a corpus having V words, the co-occurrence matrix X will be a V x V   matrix, where the i th row and j th column of X, X_ij denotes how many times word i has co-occurred with word j. An example co-occurrence matrix might look as follows.
 </p>
 <br>
    <img src="https://miro.medium.com/max/347/1*QWcK8CIDs8kMkOwsOxvywA.png",width="500" height="600" class="center" style="margin-left: auto;
  margin-right: auto;align:center">
  
 <p style="font-size:12px; font-family:monospace;text-align:center;">The co-occurrence matrix for the sentence “the cat sat on the mat” with a window size of 1. As you probably noticed it is a symmetric matrix.</p>

<h1 style="color: #484848;font-size: 20px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">How to preprocess text-data for Glove embeddings?</span></h1>

<p style="font-size:16px;  font-family:monospace;">
    When we train our model by glove embeddings we should not follow usual apparoach for pre-processing because by doing that we can lost useful informations. target should be<span style='font-weight:bold'> get our vocabulary as close to the embeddings as possible.</span>Getting your vocabulary close to the pretrained embeddings means, that you should aim for your preprocessing,to result in tokens that are mostly covered by word vectors. 
    <br>
    "Each pretrained embedding needs its own preprocessing"
    <br>
    If people used different preprocessing for training their embeddings you would also need to do the same,Especially point to can be quite challenging, if you want to concatenate embeddings as in this kernel. Imagine Embedding A preprocesses "don't" to a single token["dont"] and Embedding B to two tokens["do","n't"]. You are basically not able to do both. So you need to find a compromise.
    </p>

In [ ]:
# these function are borrowed from:-"https://www.kaggle.com/christofhenkel/how-to-preprocessing-for-glove-part1-eda"

import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr

In [ ]:
CRAWL_EMBEDDING_PATH = '../input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl'
GLOVE_EMBEDDING_PATH = '../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'

In [ ]:
# load embeddings
tic = time.time()
glove_embeddings = load_embeddings(GLOVE_EMBEDDING_PATH)
print(HEAD+f'loaded {len(glove_embeddings)} word vectors in {time.time()-tic}s')

<blockquote><h1 style="color: #484848;font-size: 20px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Check coverage without any preprocessing</span></h1></blockquote>

In [ ]:
vocab = build_vocab(list(train_df['comment_text'].apply(lambda x:x.split())))
oov = check_coverage(vocab,glove_embeddings)
for i in oov[:10]:
    print(COMMENT2+f"{i}")

<blockquote><h1 style="color: #484848;font-size: 20px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Check coverage with standard preprocessing</span></h1></blockquote>

In [ ]:
vocab = build_vocab(list(train_df['simple_preprocess'].apply(lambda x:x.split())))
oov = check_coverage(vocab,glove_embeddings)
for i in oov[:10]:
    print(COMMENT1+f"{i}")

<blockquote><p style="color: #484848;font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">As we can see only 27.78% embeddings match to vocab of comments without any pre-processing and only 38.06% to comments with standard pre-processing.</span></p></blockquote>

<p style="color: #484848;font-size: 20px;font-family: monospace; letter-spacing: 2px;cursor: pointer"> Lets explore the embeddings, in particular symbols a bit. For that we first need to define "what is a symbol" in contrast to a regular letter.</p>

In [ ]:
latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
white_list = string.ascii_letters + string.digits + latin_similar + ' '
white_list += "'"

<blockquote><p style="color: #484848;font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Print all the symbols that we have an embeddings for.We can delete all symbols that we do not have embeddings.</span></p></blockquote>

In [ ]:
glove_chars = ''.join([c for c in tqdm(glove_embeddings) if len(c) == 1])
glove_symbols = ''.join([c for c in glove_chars if not c in white_list])
glove_symbols

<blockquote><p style="color: #484848;font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Lets print all the symbols that are presented in our dataset.</span></p></blockquote>

In [ ]:
jigsaw_chars = build_vocab(list(train_df["comment_text"]))
jigsaw_symbols = ''.join([c for c in jigsaw_chars if not c in white_list])
jigsaw_symbols


<blockquote><p style="color: #484848;font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Print symbols that are presented in dataset but we do not have embeddings for.</span></p></blockquote>

In [ ]:
symbols_to_delete = ''.join([c for c in jigsaw_symbols if not c in glove_symbols])
symbols_to_delete

<blockquote><p style="color: #484848;font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Print symbols that are presented in dataset and we also have embeddings for.</span></p></blockquote>

In [ ]:
symbols_to_isolate = ''.join([c for c in jigsaw_symbols if c in glove_symbols])
symbols_to_isolate


In [ ]:
%%time
tqdm.pandas()
train_df['comment_text'] = train_df['comment_text'].progress_apply(lambda x:remove_punctuation(x))
test_df['comment_text'] = test_df['comment_text'].progress_apply(lambda x:remove_punctuation(x))

<p style="color: #484848;font-size: 20px;font-family: monospace; letter-spacing: 2px;cursor: pointer"> Now we build the function that will handle punctuation by removing all the symbols that we do not have embeddings for and isolating the symbols that we have embeddings for. </p>

In [ ]:
isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}


def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

In [ ]:
train_df['comment_text'] = train_df['comment_text'].progress_apply(lambda x:handle_punctuation(x))
test_df['comment_text'] = test_df['comment_text'].progress_apply(lambda x:handle_punctuation(x))

<blockquote><p style="color: #484848;font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Print result of fuction for handling punctuation and see coverage after removing symbols that we do not have embeddings for
</span></p></blockquote>

In [ ]:
vocab = build_vocab(list(train_df['comment_text'].apply(lambda x:x.split())))
oov = check_coverage(vocab,glove_embeddings)
for i in oov[:10]:
    print(COMMENT2+f"{i}")

<p style="color: #484848;font-size: 20px;font-family: monospace; letter-spacing: 2px;cursor: pointer"> Now we can see in previus output data contains words with <span style="font-weight:bold">appstrophe </span>that we need to remove because <span style="font-weight:bold">isn't</span> and <span style="font-weight:bold">is not</span>
are different words for our word vector.</p>

In [ ]:
train_df['comment_text'] = train_df['comment_text'].progress_apply(lambda x:remove_apostrophe(x))
test_df['comment_text'] = test_df['comment_text'].progress_apply(lambda x:remove_apostrophe(x))

In [ ]:
vocab = build_vocab(list(train_df['comment_text'].apply(lambda x:x.split())),verbose=False)
oov = check_coverage(vocab,glove_embeddings)
for i,j in zip(oov[:10],oov[10:20]):
    print(COMMENT2+f"{i}")
    print(COMMENT1+f"{j}")

In [ ]:
def get_top_n_words(corpus, n=None):
    tfidf = TfidfVectorizer()
    tfidf = tfidf.fit(train_df['comment_text'])
    bag_of_words = tfidf.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0,idx]) for word, idx in tfidf.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

def get_top_n_bigram(corpus, n=None):
    tfidf = TfidfVectorizer(ngram_range=(2,2))
    tfidf = tfidf.fit(train_df['comment_text'])
    bag_of_words = tfidf.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0,idx]) for word, idx in tfidf.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

def get_top_n_trigram(corpus, n=None):
    tfidf = TfidfVectorizer(ngram_range=(3,3))
    tfidf = tfidf.fit(train_df['comment_text'])
    bag_of_words = tfidf.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0,idx]) for word, idx in tfidf.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# functions for ploting the barchat

def plot_bt(fn,corpus,color,title):
    common_words = fn(corpus,10)
    common_words_df = DataFrame (common_words, columns=['word','freq'])
    
    colors =[]
    for cols in color:
        colors.extend([cols for i in range(2)])
        
    fig = go.Figure(go.Bar(
        x = common_words_df['freq'],
        y = common_words_df['word'],
        marker_color = colors,
        orientation='h',
    ))
    fig.update_traces(
        marker_line_color = 'black',
        marker_line_width = 1,
        opacity=0.9,
    )
    fig.update_layout(
        paper_bgcolor='#234E70',
        plot_bgcolor='rgba(0,0,0,0)',
        title="Top 10 "+title[0]+" of "+title[1],
        yaxis = {'categoryorder':'total ascending'},
        xaxis = dict(side="top", zerolinecolor = "#4d4d4d", zerolinewidth = 1, gridcolor="#e7e7e7"),
        font=dict(color='white',family="Courier New, monospace"),
    )
    fig.show()
    return common_words_df

In [ ]:
color_BuGn = px.colors.sequential.BuGn
color_Blues = px.colors.sequential.Blues
color_BuPu = px.colors.sequential.BuPu
color_OrRd = px.colors.sequential.OrRd
color_PuBu = px.colors.sequential.PuBu
color_RdPu = px.colors.sequential.RdPu
color_YlGn = px.colors.sequential.YlGn
color_deep = px.colors.sequential.deep
color_Tropic = px.colors.sequential.Purp

corpus_toxic = train_df['comment_text'][train_df.toxic==1]
corpus_severe_toxic = train_df['comment_text'][train_df.severe_toxic==1]
corpus_obscene = train_df['comment_text'][train_df.obscene==1]
corpus_threat = train_df['comment_text'][train_df.threat==1]
corpus_insult = train_df['comment_text'][train_df.insult==1]
corpus_identity_hate = train_df['comment_text'][train_df.identity_hate==1]
corpus_clean = train_df['comment_text'][train_df.none==1]

<div class="alert alert-block alert-info" style="font-size:14px; font-family:cursive;">
    <h1 style="color: #484848;font-size: 35px;font-weight: bold;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear"><center>Unigram for all labels</center></span></h1>
</div>

In [ ]:
common_unigram_clean = plot_bt(get_top_n_words, corpus_clean,color_YlGn,["unigram", "clean comments"])
common_unigram_toxic = plot_bt(get_top_n_words,corpus_toxic,color_BuGn,["unigram","toxic comments"])
common_unigram_severe_toxic = plot_bt(get_top_n_words,corpus_severe_toxic,color_Blues,["unigram","severe toxic comments"])
common_unigram_obscene = plot_bt(get_top_n_words, corpus_obscene,color_BuPu, ["unigram","obscene comments"])
common_unigram_threat = plot_bt(get_top_n_words, corpus_threat,color_OrRd,["unigram","threat comments"])
common_unigram_insult = plot_bt(get_top_n_words, corpus_insult, color_PuBu,["unigram","insult comments"])
common_unigram_identity_hate = plot_bt(get_top_n_words,corpus_identity_hate,color_RdPu,["unigram","identity hate comments"])

<div class="alert alert-block alert-info" style="font-size:14px; font-family:cursive;">
    <h1 style="color: #484848;font-size: 35px;font-weight: bold;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear"><center>Bigram for all labels</center></span></h1>
</div>

In [ ]:
common_bigram_clean = plot_bt(get_top_n_bigram, corpus_clean,color_YlGn,["bigram", "clean comments"])
common_bigram_toxic = plot_bt(get_top_n_bigram,corpus_toxic,color_BuGn,["bigram","toxic comments"])
common_bigram_severe_toxic = plot_bt(get_top_n_bigram,corpus_severe_toxic,color_Blues,["bigram","severe toxic comments"])
common_bigram_obscene = plot_bt(get_top_n_bigram, corpus_obscene,color_BuPu, ["bigram","obscene comments"])
common_bigram_threat = plot_bt(get_top_n_bigram, corpus_threat,color_OrRd,["bigram","threat comments"])
common_bigram_insult = plot_bt(get_top_n_bigram, corpus_insult, color_PuBu,["bigram","insult comments"])
common_bigram_identity_hate = plot_bt(get_top_n_bigram,corpus_identity_hate,color_RdPu,["bigram","identity hate comments"])

<div class="alert alert-block alert-info" style="font-size:14px; font-family:cursive;">
    <h1 style="color: #484848;font-size: 35px;font-weight: bold;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear"><center>Trigram for all labels</center></span></h1>
</div>

In [ ]:
common_trigram_clean = plot_bt(get_top_n_trigram, corpus_clean,color_YlGn,["trigram", "clean comments"])
common_trigram_toxic = plot_bt(get_top_n_trigram,corpus_toxic,color_BuGn,["trigram","toxic comments"])
common_trigram_severe_toxic = plot_bt(get_top_n_trigram,corpus_severe_toxic,color_Blues,["trigram","severe toxic comments"])
common_trigram_obscene = plot_bt(get_top_n_trigram, corpus_obscene,color_BuPu, ["trigram","obscene comments"])
common_trigram_threat = plot_bt(get_top_n_trigram, corpus_threat,color_OrRd,["trigram","threat comments"])
common_trigram_insult = plot_bt(get_top_n_trigram, corpus_insult, color_PuBu,["trigram","insult comments"])
common_trigram_identity_hate = plot_bt(get_top_n_trigram,corpus_identity_hate,color_RdPu,["trigram","identity hate comments"])

<h1 style="color: #484848;font-size: 20px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Configuring TPU's</span></h1>

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

<h1 style="color: #484848;font-size: 20px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Pre-process test data</span></h1>

In [ ]:
def preprocess(x):
    x = remove_punctuation(x)
    x = handle_punctuation(x)
    x = remove_apostrophe(x)
    return x

test_df['comment_text'] = test_df['comment_text'].progress_apply(lambda x:preprocess(x))

In [ ]:
train_x = train_df['comment_text']
test_x = test_df['comment_text']
train_y = train_df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]

<h1 style="color: #484848;font-size: 15px;font-weight: bold;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">The below function is the most importent one yet,in this function we building a matrix for pre-trained embeddings and with that also match our words with embeddings for both upper and lower cases.</span></h1>

In [ ]:
# This function is borrowed from "https://www.kaggle.com/christofhenkel/how-to-preprocessing-for-glove-part1-eda"

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((max_features + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        if i <= max_features:
            try:
                embedding_matrix[i] = embedding_index[word]
            except KeyError:
                try:
                    embedding_matrix[i] = embedding_index[word.lower()]
                except KeyError:
                    try:
                        embedding_matrix[i] = embedding_index[word.title()]
                    except KeyError:
                        unknown_words.append(word)
    return embedding_matrix, unknown_words


<h1 style="color: #484848;font-size: 25px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Tokenization</span></h1>
<p style="color: #484848;font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">We need to perform tokenization - the processing of segmenting text into sentences of words. The benefit of tokenization is that it gets the text into a format that is easier to convert to raw numbers, which can actually be used for processing.</span></p>

In [ ]:
max_features=400000
tokenizer = Tokenizer(num_words=max_features,filters='',lower=False)
tokenizer.fit_on_texts(list(train_x)+list(test_x))

crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): ', len(unknown_words_crawl))

glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

max_features = max_features or len(tokenizer.word_index) + 1
max_features

embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
embedding_matrix.shape

del crawl_matrix
del glove_matrix
gc.collect()

<h1 style="color: #484848;font-size: 25px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">
Sequence Bucketing</span></h1>

In [ ]:
train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

maxlen = 300
x_train_padded = pad_sequences(train_x, maxlen=maxlen)
x_test_padded = pad_sequences(test_x, maxlen=maxlen)
x_train_padded.shape

<a id='7'></a>
<h1 style="color: #484848;font-size: 35px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Training🏋️</span></h1>

<a id='7.1'></a>
<h2 style="color: #484848;font-size: 25px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer" ><span style=" transition: .5s linear">Long Short Term Memory networks –</span></h2>
<a style="color: #484848;font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer" href="https://colah.github.io/posts/2015-08-Understanding-LSTMs/"><span style=" transition: .5s linear">Long Short Term Memory networks – usually just called “LSTMs” – are a special kind of RNN, capable of learning long-term dependencies. They were introduced by Hochreiter & Schmidhuber (1997), and were refined and popularized by many people in following work. They work tremendously well on a large variety of problems, and are now widely used.</span></a>
<br>
<br>

<p style="text-align:right;font-style:monospace;font-size:11px">If you have no idea about what LSTM is i suggest you to click above text.</p>


In [ ]:
%%time
with strategy.scope():
    model = Sequential()
    
    model.add(Embedding(
        input_dim=embedding_matrix.shape[0], 
        output_dim=embedding_matrix.shape[1], 
        weights = [embedding_matrix], 
        input_length=maxlen
    ))
    
    model.add(Bidirectional(LSTM(
        maxlen, 
        return_sequences = True, 
    )))
    
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(maxlen, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(maxlen, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation = 'sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
# I will only run 2 epochs
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'model.h5', 
    monitor = 'val_loss', 
    verbose = 1, 
    save_best_only = True
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss', 
    factor = 0.2, 
    verbose = 1, 
    patience = 2,                        
    min_lr = 0.005
)
history = model.fit(
    x_train_padded, 
    train_y, 
    epochs = 2,
    batch_size = 128,
    validation_split=0.2,
    verbose = 1,
    callbacks = [reduce_lr, checkpoint]
 )

In [ ]:
load_model1 = load_model('./model.h5')

In [ ]:
prediction= load_model1.predict(x_test_padded)

In [ ]:
unzip = zipfile.ZipFile('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
unzip.extractall()
subm = pd.read_csv('./sample_submission.csv')

In [ ]:
subm[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = prediction

subm.to_csv("sub_predictions8.csv", index=False)

<h1 style="color: #484848;font-size: 35px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer"><span style=" transition: .5s linear">Refrences</span></h1>



<a style="font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer" href="https://colah.github.io/posts/2015-08-Understanding-LSTMs/">1.Understanding LSTM Neworks</a>

<a style="font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer" href="https://www.kaggle.com/christofhenkel/how-to-preprocessing-for-glove-part1-eda">2.How To: Preprocessing for GloVe Part1: EDA</a>

<a style="font-size: 15px;font-weight: bold;padding:30px;font-family: monospace; letter-spacing: 2px;cursor: pointer" href="https://towardsdatascience.com/light-on-math-ml-intuitive-guide-to-understanding-glove-embeddings-b13b4f19c010">3.Intuitive Guide to Understanding GloVe Embeddings</a>



<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:110%;
           font-family:Verdana;
           letter-spacing:0.5px">
        <p style="padding: 10px;
              color:white;font-size:20px">
            Thanks for your time,And If you like this notebook please consider a upvote.
        </p>
    </div>